# Model Building - Support Vector Machine

This notebook implements the model building for my thesis comparing different supervised machine learning techniques (logistic regression, random forests, support vector machines and neural networks). The main focus surrounds when each method results in better performance as measured by discrimination (AUC) and calibration (calibration plots). This project considers various factors including the number of features, number of datapoints and 'predictor strength'. This notebook fits a Support Vector Machine to a small number of variables and outputs prediction and results to a pandas dataframe, where model evaliation and results will be undertaken in a following notebook.

In [1]:
import glob
import numpy as np
import os
import pandas as pd
import sklearn

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, train_test_split
from scipy.stats import loguniform, uniform, randint

np.random.seed(21)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
def hyperparameter_randomiser_svm(c_range,
                                  gamma_range,
                                  c_dist = "log_uniform",
                                  gamma_dist = "log_uniform",
                                  prints = False):
    '''
    Returns the best hyperparameters from given ranges using a random search algorithm:
    c_range: Upper and lower bounds of the c distribution range as a list
    gamma_range: Upper and lower bounds of the gamma distribution range as a list
    c_dist: An argument specifying whether the distribution is drawn from a uniform or log uniform distribution
            arguments: "log_uniform", "uniform"
    gamma_dist: An argument specifying whether the distribution is drawn from a uniform or log uniform distribution
                arguments: "log_uniform", "uniform"
    prints: An option to facilitate the printing of the randomly selected hyperparameters at each search.
    '''
    if c_dist=="log_uniform":
        c = loguniform(c_range[0], c_range[1]).rvs(1).item()
    elif c_dist=="uniform":
        c = uniform(c_range[0], c_range[1]).rvs(1).item()
    else:
        raise Exception("Please provide a valid c distribution.")
    if gamma_dist=="log_uniform":
        gamma = loguniform(gamma_range[0], gamma_range[1]).rvs(1).item()
    elif gamma_dist=="uniform":
        gamma = uniform(gamma_range[0], gamma_range[1]).rvs(1).item()
    else:
        raise Exception("Please provide a valid gamma distribution.")
    if prints == True:
        print("c:", c)
        print("gamma:", gamma)
    return c, gamma

In [3]:
def build_model_svm(ce, gamma):
    SVM = SVC(C = ce, gamma = gamma, kernel = 'rbf', probability = True)
    return SVM

In [4]:
def hyp_tuner_svm(cee_range,
                  gam_range,
                  X_train, 
                  y_train, 
                  X_test, 
                  y_test,
                  cv_folds = 5, 
                  random_search = 20):
    '''Returns the best hyperparameters from given ranges using a random search algorithm:
        c_range: Range of max depth of consituent decision trees
        gamma_range: Range of max features in each split
        X_train: The training dataset (as a pandas dataframe) features
        y_train: The training dataset (as a pandas dataframe) labels
        X_test: The testing dataset (as a pandas dataframe) features
        y_test: The testing dataset (as a pandas dataframe) labels
        cv_folds: Number of folds of cross validation (default 5)
        random_search: how many random searches are performed (default 20)
        '''
    cv = StratifiedKFold(n_splits = cv_folds, shuffle = True, random_state = 21)
    cv.get_n_splits(X_train, y_train)
    #splitting the dataset for k-fold
    c_list, gamma_list = [], []
    auc_dict = dict()
    #creating dictionaries and lists that store information on hyperparameters and aucs
    for i in range(random_search):
        print("Running search {counter} out of {maxi}".format(counter = str(i+1), maxi = str(random_search)))
        #randomly selecting hyperparameters and storing them
        cee, gam = hyperparameter_randomiser_svm(c_range = cee_range,
                                                 gamma_range = gam_range) 
        c_list.append(cee)
        gamma_list.append(gam)
        auc_dict[i] = []
        for train_index, test_index in cv.split(X_train, y_train):
            #cross validating
            model = build_model_svm(ce = cee,
                                    gamma = gam)
            X1_train, X1_valid = X_train.iloc[train_index], X_train.iloc[test_index]
            y1_train, y1_valid = y_train.iloc[train_index], y_train.iloc[test_index]
            model.fit(X1_train, y1_train)
            y1_pred = model.predict_proba(X1_valid)[:, 1]
            fpr, tpr, threshold = roc_curve(y1_valid, y1_pred)
            cauc = auc(fpr, tpr)
            auc_dict[i].append(cauc)
    #analysing results to find best hyperparameters
    hp_dict = {"c": c_list,
               "gamma": gamma_list}
    av_auc_dict = dict()
    for k, v in auc_dict.items():
        av_auc_dict[k] = sum(v)/len(v)
    opt_hyper = max(av_auc_dict, key = av_auc_dict.get)
    opt_c = hp_dict["c"][opt_hyper]
    opt_gamma = hp_dict["gamma"][opt_hyper]
    #building the model based on the whole training data based on optimal parameters
    opt_model = build_model_svm(ce = opt_c, 
                                gamma = opt_gamma)
    opt_model.fit(X_train, y_train)
    y_pred = opt_model.predict_proba(X_test)[:, 1]
    v_auc = roc_auc_score(y_test, y_pred)
    #sorting prediction df
    y_test_df = pd.DataFrame(y_test)
    pred_df = pd.DataFrame(y_pred)
    pred_df['index'] = y_test.index
    pred_df.set_index('index', inplace = True)
    pred_df.rename(columns = {0: "prob"}, inplace = True)
    df = pred_df.join(y_test_df, how = 'left')
    print("The best hyperparameters were:")
    print("c:", opt_c)
    print("gamma:", opt_gamma)
    y_pred2 = opt_model.predict(X_test)
    print(confusion_matrix(y_test, y_pred2))
    opt_svm = opt_model
    
    return df, v_auc

In [5]:
def min_max_scaling(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm
# credit:
# https://towardsdatascience.com/data-normalization-with-pandas-and-scikit-learn-7c1cc6ed6475#:~:text=maximum%20absolute%20scaling.-,The%20min%2Dmax%20feature%20scaling,max()%20methods.

In [6]:
current_wd = os.getcwd()
base_wd = current_wd
base_wd = base_wd.replace("\\", "/")
path = base_wd + "/data/bootstraps"
os.chdir(path)
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

In [7]:
%%time
#fitting support vector machines to each bootstrap
np.random.seed(21)
i = 0
dataframes = []
for filename in all_filenames:
    i += 1
    print(str(i) + " of " + str(len(all_filenames)))
    mf = pd.read_csv(filename)
    df = min_max_scaling(mf)
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(df.drop('deceased', axis = 1), 
                                                                        df['deceased'], test_size = 0.2, 
                                                                        random_state = 21, 
                                                                        stratify = df['deceased'])
    preds, sv_auc = hyp_tuner_svm(cee_range = [0.001, 1000],
                                  gam_range = [0.001, 1000],
                                  X_train = X_train,
                                  y_train = y_train,
                                  X_test = X_test,
                                  y_test = y_test,
                                  cv_folds = 3,
                                  random_search = 5)
    preds.reset_index(drop = True, inplace = True)
    dataframes.append(preds)
    print(sv_auc)
svm_predictions = pd.concat(dataframes, axis = 1)
svm_predictions.to_csv('./predictions/sv_predictions.csv', index = False)

1 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.9453166319420468
gamma: 152.12404587716756
[[823   2]
 [ 59  68]]
0.9598950131233595
2 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 179.71136701341393
gamma: 362.06540171441236
[[817   0]
 [ 60  75]]
0.9488598757876604
3 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 4.001433394614622
gamma: 245.49464966698156
[[820   1]
 [ 64  67]]
0.9524690611895753
4 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 23.266076729072953
gamma: 0.22503410581264216
[[

The best hyperparameters were:
c: 503.8276626568869
gamma: 0.032921337878991264
[[812   9]
 [110  21]]
0.8487322293609543
33 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 14.19361706080262
gamma: 21.544805142538237
[[782  34]
 [ 43  93]]
0.9032538566897347
34 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 2.514549321707827
gamma: 0.02746541422292303
[[824   0]
 [128   0]]
0.8409530491504855
35 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.5610521780053135
gamma: 179.3622354749524
[[820   0]
 [ 96  36]]
0.9473300073909832
36 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running s

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 197.1094981717769
gamma: 41.83988228910064
[[804  21]
 [ 39  88]]
0.9351944643283228
65 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.027094124033178537
gamma: 115.5384819123988
[[821   0]
 [131   0]]
0.947429591542617
66 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.016220420012901544
gamma: 363.49139629314476
[[813   0]
 [139   0]]
0.9509189696213509
67 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.7283610430775547
gamma: 337.3783671133212
[[813   0]
 [112  27]]
0.9461183820

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 51.40954622727737
gamma: 127.14983521593882
[[811  10]
 [ 48  83]]
0.9430037842511924
97 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.0211321386529459
gamma: 0.11951907282075243
[[814   0]
 [138   0]]
0.835843748887227
98 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.12865853664543753
gamma: 25.294951713615788
[[822   0]
 [129   1]]
0.8887703537338575
99 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.14302570036986992
gamma: 147.42389536837712
[[823   0]
 [129   0]]
0.93832358

Running search 5 out of 5
The best hyperparameters were:
c: 79.81904713162766
gamma: 404.13624443790906
[[819   1]
 [ 64  68]]
0.9536123429416112
128 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 2.1194793146070205
gamma: 45.990902920427494
[[814  11]
 [ 54  73]]
0.900548795036984
129 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.0020985920523385175
gamma: 286.7139628891868
[[822   0]
 [130   0]]
0.9511884708965002
130 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.6874051851879138
gamma: 471.0716831913715
[[821   0]
 [105  26]]
0.9447982817454045
131 of 999
Running search 1 out of 5
Running search 2 out of 5
Runni

...

Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 147.74156270030412
gamma: 12.718469200067604
[[793  33]
 [ 45  81]]
0.8638302778738615
851 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.19240640294576744
gamma: 300.9047417271667
[[820   0]
 [132   0]]
0.9568043237250554
852 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 457.1639148201465
gamma: 211.33868579623254
[[815   6]
 [ 50  81]]
0.9561696311517326
853 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 108.24867782170651
gamma: 1.6211025028193444
[[808  20]
 [ 77  47]]
0.8252103786816269
854 of 999
Runni

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 236.99717761051713
gamma: 0.002876543652010474
[[816   0]
 [136   0]]
0.7979112600922722
883 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.011623907421646183
gamma: 54.84439299504926
[[813   0]
 [139   0]]
0.9323935685400018
884 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 9.921434753544633
gamma: 132.8940808148305
[[817   6]
 [ 51  78]]
0.9567285503028248
885 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 478.6517115015831
gamma: 0.010023374371268087
[[820   4]
 [115  13]]
0.8543

Running search 5 out of 5
The best hyperparameters were:
c: 2.532878073807745
gamma: 418.32494625348346
[[821   0]
 [ 54  77]]
0.9520831977387472
914 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.15968870966240759
gamma: 123.43894915594586
[[821   0]
 [131   0]]
0.9291777854227297
915 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 8.565500668827864
gamma: 0.004149299054463459
[[819   0]
 [133   0]]
0.8679207175447778
916 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 353.965570500726
gamma: 0.004318070813058466
[[817   0]
 [135   0]]
0.8392764857881138
917 of 999
Running search 1 out of 5
Running search 2 out of 5
Run

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.10428783384967218
gamma: 292.18344649975495
[[824   0]
 [128   0]]
0.9459098149271845
946 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 426.12942577711556
gamma: 472.7944525003953
[[816   0]
 [ 58  78]]
0.9546118079584776
947 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.032750922533399764
gamma: 440.30747472152484
[[824   0]
 [128   0]]
0.9447815533980581
948 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.0013888020793214673
gamma: 260.9082585791644
[[825   0]
 [127   0]]
0.87

Running search 5 out of 5
The best hyperparameters were:
c: 0.01100355718038945
gamma: 0.001753922910209812
[[821   0]
 [131   0]]
0.7592026108543852
977 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 0.12413496002935116
gamma: 75.33984005670852
[[816   0]
 [136   0]]
0.9239024653979239
978 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 3.9676371254555316
gamma: 117.47539100964373
[[820  10]
 [ 56  66]]
0.9360211337151885
979 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
c: 415.81755854994975
gamma: 115.78923664621301
[[812   9]
 [ 52  79]]
0.9391358518284348
980 of 999
Running search 1 out of 5
Running search 2 out of 5
R